In [1]:
!cp -r ../input/d/romanofrancesco/recsys-repo/RecSys_Course_AT_PoliMi-master/* ./

In [2]:
%config Completer.use_jedi = False
import pandas as pd
import numpy as np
import scipy.sparse as sps
import matplotlib.pyplot as pyplot
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_path = "../input/recommender-system-2021-challenge-polimi/data_train.csv"
URM_all_dataframe = pd.read_csv(filepath_or_buffer=URM_path, 
                                sep=",",
                                dtype={0:int, 1:int, 2:float},
                                header=0)
URM_all_dataframe.columns = ["UserID", "ItemID", "Interaction"]

userID_unique = URM_all_dataframe["UserID"].unique()
itemID_unique = URM_all_dataframe["ItemID"].unique()

n_users = len(userID_unique)
n_items = len(itemID_unique)
n_interactions = len(URM_all_dataframe)

URM_all = sps.coo_matrix((URM_all_dataframe["Interaction"].values, 
                          (URM_all_dataframe["UserID"].values, URM_all_dataframe["ItemID"].values)))
URM_all = URM_all.tocsr() # to obtain fast access to rows (users)

In [3]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

# split data into train and validation data 80/20
URM_train, URM_valid = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

In [4]:
from Evaluation.Evaluator import EvaluatorHoldout

#create an evaluator object to evaluate validation set
#we will use it for hyperparameter tuning
evaluator_valid = EvaluatorHoldout(URM_valid, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13642 ( 0.1%) Users that have less than 1 test interactions


In [5]:
import numpy as np
import scipy.sparse as sps
from Recommenders.Recommender_utils import check_matrix
from sklearn.linear_model import ElasticNet
from Recommenders.BaseSimilarityMatrixRecommender import BaseItemSimilarityMatrixRecommender
from Utils.seconds_to_biggest_unit import seconds_to_biggest_unit
import time, sys
from tqdm import tqdm
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

# os.environ["PYTHONWARNINGS"] = ('ignore::exceptions.ConvergenceWarning:sklearn.linear_model')
# os.environ["PYTHONWARNINGS"] = ('ignore:Objective did not converge:ConvergenceWarning:')

class SLIMElasticNetRecommender(BaseItemSimilarityMatrixRecommender):
    """
    Train a Sparse Linear Methods (SLIM) item similarity model.
    NOTE: ElasticNet solver is parallel, a single intance of SLIM_ElasticNet will
          make use of half the cores available
    See:
        Efficient Top-N Recommendation by Linear Regression,
        M. Levy and K. Jack, LSRS workshop at RecSys 2013.
        SLIM: Sparse linear methods for top-n recommender systems,
        X. Ning and G. Karypis, ICDM 2011.
        http://glaros.dtc.umn.edu/gkhome/fetch/papers/SLIM2011icdm.pdf
    """

    RECOMMENDER_NAME = "SLIMElasticNetRecommender"

    def __init__(self, URM_train, verbose = True):
        super(SLIMElasticNetRecommender, self).__init__(URM_train, verbose = verbose)

    @ignore_warnings(category=ConvergenceWarning)
    def fit(self, l1_ratio=0.1, alpha = 1.0, positive_only=True, topK = 100,**earlystopping_kwargs):

        assert l1_ratio>= 0 and l1_ratio<=1, "{}: l1_ratio must be between 0 and 1, provided value was {}".format(self.RECOMMENDER_NAME, l1_ratio)

        self.l1_ratio = l1_ratio
        self.positive_only = positive_only
        self.topK = topK


        # initialize the ElasticNet model
        self.model = ElasticNet(alpha=alpha,
                                l1_ratio=self.l1_ratio,
                                positive=self.positive_only,
                                fit_intercept=False,
                                copy_X=False,
                                precompute=True,
                                selection='random',
                                max_iter=100,
                                tol=1e-4)

        URM_train = check_matrix(self.URM_train, 'csc', dtype=np.float32)

        n_items = URM_train.shape[1]

        # Use array as it reduces memory requirements compared to lists
        dataBlock = 10000000

        rows = np.zeros(dataBlock, dtype=np.int32)
        cols = np.zeros(dataBlock, dtype=np.int32)
        values = np.zeros(dataBlock, dtype=np.float32)

        numCells = 0

        start_time = time.time()
        start_time_printBatch = start_time

        # fit each item's factors sequentially (not in parallel)
        for currentItem in range(n_items):

            # get the target column
            y = URM_train[:, currentItem].toarray()

            # set the j-th column of X to zero
            start_pos = URM_train.indptr[currentItem]
            end_pos = URM_train.indptr[currentItem + 1]

            current_item_data_backup = URM_train.data[start_pos: end_pos].copy()
            URM_train.data[start_pos: end_pos] = 0.0

            # fit one ElasticNet model per column
            self.model.fit(URM_train, y)

            # self.model.coef_ contains the coefficient of the ElasticNet model
            # let's keep only the non-zero values

            # Select topK values
            # Sorting is done in three steps. Faster then plain np.argsort for higher number of items
            # - Partition the data to extract the set of relevant items
            # - Sort only the relevant items
            # - Get the original item index

            nonzero_model_coef_index = self.model.sparse_coef_.indices
            nonzero_model_coef_value = self.model.sparse_coef_.data

            local_topK = min(len(nonzero_model_coef_value)-1, self.topK)

            relevant_items_partition = (-nonzero_model_coef_value).argpartition(local_topK)[0:local_topK]
            relevant_items_partition_sorting = np.argsort(-nonzero_model_coef_value[relevant_items_partition])
            ranking = relevant_items_partition[relevant_items_partition_sorting]

            for index in range(len(ranking)):

                if numCells == len(rows):
                    rows = np.concatenate((rows, np.zeros(dataBlock, dtype=np.int32)))
                    cols = np.concatenate((cols, np.zeros(dataBlock, dtype=np.int32)))
                    values = np.concatenate((values, np.zeros(dataBlock, dtype=np.float32)))


                rows[numCells] = nonzero_model_coef_index[ranking[index]]
                cols[numCells] = currentItem
                values[numCells] = nonzero_model_coef_value[ranking[index]]

                numCells += 1

            # finally, replace the original values of the j-th column
            URM_train.data[start_pos:end_pos] = current_item_data_backup

            elapsed_time = time.time() - start_time
            new_time_value, new_time_unit = seconds_to_biggest_unit(elapsed_time)


            if time.time() - start_time_printBatch > 300 or currentItem == n_items-1:
                self._print("Processed {} ({:4.1f}%) in {:.2f} {}. Items per second: {:.2f}".format(
                    currentItem+1,
                    100.0* float(currentItem+1)/n_items,
                    new_time_value,
                    new_time_unit,
                    float(currentItem)/elapsed_time))

                sys.stdout.flush()
                sys.stderr.flush()

                start_time_printBatch = time.time()

        # generate the sparse weight matrix
        self.W_sparse = sps.csr_matrix((values[:numCells], (rows[:numCells], cols[:numCells])),
                                       shape=(n_items, n_items), dtype=np.float32)

In [ ]:
recommender_SLIMElasticNet = SLIMElasticNetRecommender(URM_train)
recommender_SLIMElasticNet.fit(epochs = 500, l1_ratio = 0.0023170159712850467, alpha = 0.09078974149197175, 
                positive_only = True, topK = 363)

In [6]:
import pyximport
pyximport.install()
!python run_compile_all_cython.py

run_compile_all_cython: Found 10 Cython files in 4 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/opt/conda/bin/python'
Compiling [1/10]: MatrixFactorization_Cython_Epoch.pyx... 
In file included from /opt/conda/lib/python3.7/site-packages/numpy/core/include/numpy/ndarraytypes.h:1822,
                 from /opt/conda/lib/python3.7/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/conda/lib/python3.7/site-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from MatrixFactorization_Cython_Epoch.c:620:
/opt/conda/lib/python3.7/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~
MatrixFactorization_Cython_Epoch.c: In function ‘__pyx_pf_32MatrixFactorization_Cyth

In [ ]:
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
recommender_SLIM_BPR_Cython = SLIM_BPR_Cython(URM_train)
recommender_SLIM_BPR_Cython.fit(epochs=650, sgd_mode = "sgd", topK = 483, lambda_i = 0.0006712905081189398, 
                lambda_j = 0.06584150350451998, learning_rate = 0.0036482363905043207)

In [ ]:
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
recommender_RP3beta = RP3betaRecommender(URM_train)
recommender_RP3beta.fit(alpha = 0.9902904484569759, beta = 0.5159273869398134, topK = 61, implicit = False)

In [ ]:
print("MAP of the starting models")

result_df, _ = evaluator_valid.evaluateRecommender(recommender_SLIMElasticNet)
print("SLIM ElasticNet - MAP: {}".format(result_df.loc[10]["MAP"]))

result_df, _ = evaluator_valid.evaluateRecommender(recommender_SLIM_BPR_Cython)
print("SLIM BPR - MAP: {}".format(result_df.loc[10]["MAP"]))

result_df, _ = evaluator_valid.evaluateRecommender(recommender_RP3beta)
print("RP3beta - MAP: {}".format(result_df.loc[10]["MAP"]))

In [7]:
from numpy import linalg as LA
from Recommenders.BaseRecommender import BaseRecommender

class DifferentLossScoresHybridRecommender(BaseRecommender):
    """ ScoresHybridRecommender
    Hybrid of three predictions scores
    R = R1*alpha + R2*beta + R3*(1-alpha-beta)
    
    Class from Dacrema exercise modified by Antonio Ercolani
    The original took as input 2 recommender

    """

    RECOMMENDER_NAME = "DifferentLossScoresHybridRecommender"


    def __init__(self, URM_train, recommender_1, recommender_2, recommender_3):
        super(DifferentLossScoresHybridRecommender, self).__init__(URM_train)

        self.URM_train = sps.csr_matrix(URM_train)
        self.recommender_1 = recommender_1
        self.recommender_2 = recommender_2
        self.recommender_3 = recommender_3
        
        
        
    def fit(self, norm, alpha = 0.5, beta = 0.5):

        self.alpha = alpha
        self.beta = beta
        self.norm = norm


    def _compute_item_score(self, user_id_array, items_to_compute):
        
        item_weights_1 = self.recommender_1._compute_item_score(user_id_array)
        item_weights_2 = self.recommender_2._compute_item_score(user_id_array)
        item_weights_3 = self.recommender_3._compute_item_score(user_id_array)

        norm_item_weights_1 = LA.norm(item_weights_1, self.norm)
        norm_item_weights_2 = LA.norm(item_weights_2, self.norm)
        norm_item_weights_3 = LA.norm(item_weights_3, self.norm)
        
        
        if norm_item_weights_1 == 0:
            raise ValueError("Norm {} of item weights for recommender 1 is zero. Avoiding division by zero".format(self.norm))
        
        if norm_item_weights_2 == 0:
            raise ValueError("Norm {} of item weights for recommender 2 is zero. Avoiding division by zero".format(self.norm))
            
        if norm_item_weights_3 == 0:
            raise ValueError("Norm {} of item weights for recommender 3 is zero. Avoiding division by zero".format(self.norm))
        
        item_weights = item_weights_1 / norm_item_weights_1 * self.alpha + item_weights_2 / norm_item_weights_2 * self.beta + item_weights_3 / norm_item_weights_3 * (1-self.alpha-self.beta)

        return item_weights

In [ ]:
recommender_object = DifferentLossScoresHybridRecommender(URM_train, recommender_SLIMElasticNet, recommender_SLIM_BPR_Cython, recommender_RP3beta)

best_model = {
    "MAP" : 0,
    "alpha" : 0,
    "beta" : 0,
    "norm" : 0
}

for norm in [1,2]:
    for alpha in np.arange(0.0, 1.1, 0.1):
        for beta in np.arange(0.0, 1.1, 0.1):
            
            #truncate digits since np.arange sometimes doesn't
            alpha = round(alpha,1)
            beta = round(beta,1)
            
            
            #discard cases in which the sum is greater than 1 
            if ( (alpha+beta) <= 1): 
                theta = round(1-alpha-beta,1)
            
                print("----")
                recommender_object.fit(norm, alpha, beta)
                result_df, _ = evaluator_valid.evaluateRecommender(recommender_object)
                print("Norm: {}, Alpha: {}, Beta: {}, Theta: {}, Result: {}".format(norm, alpha, beta, 1-alpha-beta, result_df.loc[10]["MAP"]))

                if result_df.loc[10]["MAP"] > best_model["MAP"]:
                    best_model["MAP"] = result_df.loc[10]["MAP"]
                    best_model["alpha"] = alpha
                    best_model["norm"] = norm

                    print("*** New best model found! ")
                    print("New best model has MAP: {} with alpha: {}, beta: {}, theta: {}, norm: {}".format(best_model["MAP"], best_model["alpha"], best_model["beta"],
                                                                                                            1-best_model["alpha"]-best_model["beta"], best_model["norm"]))

print("----")
print("Best model has MAP: {} with alpha: {}, beta: {}, norm: {}".format(best_model["MAP"], best_model["alpha"], best_model["beta"], best_model["norm"]))

In [ ]:
#re-tune - Norm: , Alpha: 0.5, Beta: 0.5, Theta: 0, Result: 0.2505692650743614
# further exploration

best_model = {
    "MAP" : 0,
    "alpha" : 0,
    "beta" : 0,
    "norm" : 0
}

for norm in [1]:
    for alpha in np.arange(0.45, 0.55, 0.01):
        for beta in np.arange(0.45, 0.55, 0.01):
            
            
            #discard cases in which the sum is greater than 1 
            if ( (alpha+beta) <= 1): 
                theta = round(1-alpha-beta,1)
            
                print("----")
                recommender_object.fit(norm, alpha, beta)
                result_df, _ = evaluator_valid.evaluateRecommender(recommender_object)
                print("Norm: {}, Alpha: {}, Beta: {}, Theta: {}, Result: {}".format(norm, alpha, beta, 1-alpha-beta, result_df.loc[10]["MAP"]))

                if result_df.loc[10]["MAP"] > best_model["MAP"]:
                    best_model["MAP"] = result_df.loc[10]["MAP"]
                    best_model["alpha"] = alpha
                    best_model["beta"] = beta
                    best_model["norm"] = norm

                    print("*** New best model found! ")
                    print("New best model has MAP: {} with alpha: {}, beta: {}, theta: {}, norm: {}".format(best_model["MAP"], best_model["alpha"], best_model["beta"],
                                                                                                            1-best_model["alpha"]-best_model["beta"], best_model["norm"]))

print("----")
print("Best model has MAP: {} with alpha: {}, beta: {}, norm: {}".format(best_model["MAP"], best_model["alpha"], best_model["beta"], best_model["norm"]))

In [ ]:
# Best model has MAP: 0.2508066736727197 with alpha: 0.48000000000000004, beta: 0.51, norm: 1
delta_MAP_improvement = best_model["MAP"] - 0.24831015633273928
delta_MAP_improvement

In [8]:
recommender_SLIMElasticNet = SLIMElasticNetRecommender(URM_all)
recommender_SLIMElasticNet.fit(epochs = 500, l1_ratio = 0.0023170159712850467, alpha = 0.09078974149197175, 
                positive_only = True, topK = 363)

SLIMElasticNetRecommender: Processed 977 ( 5.4%) in 5.00 min. Items per second: 3.25
SLIMElasticNetRecommender: Processed 1967 (10.9%) in 10.01 min. Items per second: 3.27
SLIMElasticNetRecommender: Processed 2966 (16.4%) in 15.01 min. Items per second: 3.29
SLIMElasticNetRecommender: Processed 3990 (22.1%) in 20.01 min. Items per second: 3.32
SLIMElasticNetRecommender: Processed 5007 (27.7%) in 25.01 min. Items per second: 3.34
SLIMElasticNetRecommender: Processed 6012 (33.3%) in 30.01 min. Items per second: 3.34
SLIMElasticNetRecommender: Processed 7024 (38.9%) in 35.02 min. Items per second: 3.34
SLIMElasticNetRecommender: Processed 8049 (44.6%) in 40.02 min. Items per second: 3.35
SLIMElasticNetRecommender: Processed 9083 (50.3%) in 45.02 min. Items per second: 3.36
SLIMElasticNetRecommender: Processed 10114 (56.0%) in 50.02 min. Items per second: 3.37
SLIMElasticNetRecommender: Processed 11165 (61.8%) in 55.02 min. Items per second: 3.38
SLIMElasticNetRecommender: Processed 12198 

In [10]:
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
recommender_SLIM_BPR_Cython = SLIM_BPR_Cython(URM_all)
recommender_SLIM_BPR_Cython.fit(epochs=650, sgd_mode = "sgd", topK = 483, lambda_i = 0.0006712905081189398, 
                lambda_j = 0.06584150350451998, learning_rate = 0.0036482363905043207)

SLIM_BPR_Recommender: Automatic selection of fastest train mode. Available RAM is 16742.00 MB (93.01%) of 18001.00 MB, required is 1304.51 MB. Using dense matrix.
Processed 13650 (100.0%) in 1.74 sec. BPR loss is 5.31E-01. Sample per second: 7865
SLIM_BPR_Recommender: Epoch 1 of 650. Elapsed time 0.90 sec
Processed 13650 (100.0%) in 1.67 sec. BPR loss is 1.87E+00. Sample per second: 8184
SLIM_BPR_Recommender: Epoch 2 of 650. Elapsed time 1.84 sec
Processed 13650 (100.0%) in 1.44 sec. BPR loss is 3.11E+00. Sample per second: 9453
SLIM_BPR_Recommender: Epoch 3 of 650. Elapsed time 2.61 sec
Processed 13650 (100.0%) in 1.29 sec. BPR loss is 4.17E+00. Sample per second: 10590
SLIM_BPR_Recommender: Epoch 4 of 650. Elapsed time 3.46 sec
Processed 13650 (100.0%) in 1.17 sec. BPR loss is 5.20E+00. Sample per second: 11621
SLIM_BPR_Recommender: Epoch 5 of 650. Elapsed time 4.34 sec
Processed 13650 (100.0%) in 1.06 sec. BPR loss is 6.13E+00. Sample per second: 12821
SLIM_BPR_Recommender: Epoch 6 

In [11]:
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
recommender_RP3beta = RP3betaRecommender(URM_all)
recommender_RP3beta.fit(alpha = 0.9902904484569759, beta = 0.5159273869398134, topK = 61, implicit = False)

In [12]:
# final hybrid recommender which combine all 3 models
recommender = DifferentLossScoresHybridRecommender(URM_all, recommender_SLIMElasticNet, recommender_SLIM_BPR_Cython, recommender_RP3beta)
recommender.fit(norm=1, alpha = 0.48, beta = 0.51)

In [13]:
test_users = pd.read_csv('../input/recommender-system-2021-challenge-polimi/data_target_users_test.csv')
test_users

,user_id
0,0
1,1
2,2
3,3
4,4
...,...
13645,13645
13646,13646
13647,13647
13648,13648


In [14]:
user_id = test_users['user_id']
recommendations = []
for user in user_id:
    recommendations.append(recommender.recommend(user,cutoff = 10))

In [15]:
for index in range(len(recommendations)):
    recommendations[index]=np.array(recommendations[index])
    
test_users['item_list']= recommendations
test_users['item_list'] = pd.DataFrame([str(line).strip('[').strip(']').replace("'","") for line in test_users['item_list']])
test_users.to_csv('submission.csv', index=False)